In [2]:
import pandas as pd
import os
import json
import pymysql
from pyecharts.charts import Graph
from pyecharts import options as opts
from pyecharts.globals import ThemeType
import re
from dominate.tags import *
import dominate
import difflib
from pyecharts.commons.utils import JsCode

In [35]:
db = pymysql.connect(host='localhost',
                     user='',
                     password='',
                     database='public')
cursor = db.cursor()
f = open("test.sql", "w", encoding="utf-8")
with open(r"C:\Users\AAAA\Downloads\nesql-db.script", "r", encoding="utf-8") as script:
    for line in script:
        if line.startswith("INSERT INTO ITEM "):
            cursor.execute(line.strip().replace("\\", "\\\\") + ";")
            f.write(str(line.strip()).replace("\\", "\\\\") + ";\n")
            # .replace("\\", "\\\\")
f.close()
db.commit()

In [10]:
f = open("test.csv", "w")
f.write("ID,IMAGE_FILE_PATH,INTERNAL_NAME,ITEM_DAMAGE,ITEM_ID,LOCALIZED_NAME,MAX_DAMAGE,MAX_STACK_SIZE,MOD_ID,NBT,TOOLTIP,UNLOCALIZED_NAME\n")
with open("test.sql", "r", encoding="utf-8") as file:
    for line in file:
        f.write(line[24:-3].replace("\'", "\"") + "\n")
f.close()

In [3]:
db = pymysql.connect(host='localhost',
                     user='',
                     password='',
                     database='public')
cursor = db.cursor()

In [4]:
cursor = db.cursor()
        # i = re.sub("§.", "", i)
        # i = i.replace("%n", "")

In [7]:
df = pd.DataFrame(columns=["quest_file_name", "quest_id",
                  "quest_name", "quest_desc", "quest_icon", "pre_quests"])
for quest_line in os.listdir("DefaultQuests/Quests/"):
    for quest in os.listdir(f"DefaultQuests/Quests/{quest_line}/"):
        que = json.load(
            open(f"DefaultQuests/Quests/{quest_line}/{quest}", "r", encoding="utf-8"))
        
        pre_quests = []
        for i in que["preRequisites:9"]:
            pre_quests.append(que["preRequisites:9"][i]["questIDLow:4"])
        
        item_id = "i~" + que["properties:10"]['betterquesting:10']["icon:10"]["id:8"].replace(
            ":", "~") + "~" + str(que["properties:10"]['betterquesting:10']["icon:10"]["Damage:2"])
        cursor.execute(
            f"select ID, INTERNAL_NAME, LOCALIZED_NAME from item where ID like '%{item_id}%'")
        icon_name = ""
        try:
            icon_name = cursor.fetchall()[0][2].encode(
                "utf-8").decode("unicode_escape")
        except:
            ...
        
        name = que["properties:10"]["betterquesting:10"]["name:8"]
        desc = que["properties:10"]["betterquesting:10"]["desc:8"]

        name = re.sub("§.", "", name)
        name = name.replace("%n", "")
        desc = re.sub("§.", "", desc)
        desc = desc.replace("%n", "")

        df.loc[len(df)] = [quest, que["questIDLow:4"],name, desc, icon_name, pre_quests]

repeats = ["AE Spatial Storage",
           "Radioisotope Thermoelectric Generator", "Fanciful Thread"]
for word in repeats:
    counter = 1
    for i in df[df["quest_name"] == word].sort_values("quest_id").index:
        df.loc[i, "quest_name"] = df.loc[i, "quest_name"] + str(counter)
        counter += 1

KeyError: 'quest_name_zh'

In [8]:
repeats = ["AE Spatial Storage",
           "Radioisotope Thermoelectric Generator", "Fanciful Thread"]
for word in repeats:
    counter = 1
    for i in df[df["quest_name"] == word].sort_values("quest_id").index:
        df.loc[i, "quest_name"] = df.loc[i, "quest_name"] + str(counter)
        counter += 1

In [9]:
for questline in os.listdir("DefaultQuests/QuestLines"):
    nodes = []
    links = []
    for quest in os.listdir(f"DefaultQuests/QuestLines/{questline}"):
        if quest == "QuestLine.json":
            continue
        que = json.load(
            open(f"DefaultQuests/QuestLines/{questline}/{quest}", "r", encoding="utf-8"))
        line = df[df["quest_id"] == que["questIDLow:4"]]

        lens = int(len(line["quest_desc"].values[0]) / 70)
        temp_str = line["quest_desc"].values[0]
        temp_list = []
        if lens <= 1:
            temp_list.append(temp_str)
        else:
            for i in range(1, lens + 1):
                o = 70
                while o < len(temp_str):
                    if temp_str[o] != " ":
                        o += 1
                    else:
                        break
                temp_list.append(temp_str[:o])
                temp_str = temp_str[o+1 : ]
        if os.path.exists(f"html/2.7.0全物品图标/2.7.0/{line["quest_icon"].values[0]}.png"):
            path = f"image://2.7.0全物品图标/2.7.0/{line["quest_icon"].values[0]}.png"
        else:
            path = ""
        new_node = {"name": f"{line["quest_name"].values[0]}",
                        "symbolSize": int(que["sizeX:3"]) * 1.3,
                        "symbol": path,
                        "x": (que["x:3"] + int(que["sizeX:3"]) / 2),
                        "y": (que["y:3"] + int(que["sizeX:3"]) / 2),
                        "tooltip": "<br/>".join(temp_list)
                        }
        nodes.append(new_node)
        pre_quests = line["pre_quests"].values[0]
        for quest_id in pre_quests:
            pre_que = df[df["quest_id"] == quest_id]
            links.append({"source": pre_que["quest_name"].values[0],
                          "target": line["quest_name"].values[0]})

    g = (
        Graph(init_opts=opts.InitOpts(width="100%",
              height="900%", bg_color="#f5f0d3", chart_id="this_chart"))
        .add("", nodes, links, is_draggable=False, edge_symbol=['circle', 'arrow'], repulsion=0, label_opts=opts.LabelOpts(position="up", is_show=False), gravity=0, layout="none", edge_symbol_size=10)
        .add_js_funcs(
            """
            chart_this_chart.on('click', function (params) {
                    if (params.dataType === 'node') {
                        navigator.clipboard.writeText(params.data.tooltip.replaceAll("<br/>", ""));
                    }
            });
        """
        )
        .render(f"html/{questline.split("-")[0]}.html")
    )
    temp = []
    counter = 0
    for i in nodes:
        if i["name"] in temp:
            print(questline.split("-")[0], i["name"], counter)
            counter += 1
        else:
            temp.append(i["name"])

<module 'difflib' from 'c:\\Users\\AAAA\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\difflib.py'>

In [29]:
quest_line_order = []
for line in open("DefaultQuests/QuestLinesOrder.txt").readlines():
    quest_line_order.append(line.split(": ")[1].strip())

In [34]:
a = ul()
    # if "html" in i:
    #     a += li(button(i.split(".html")[0], id=i))
for que_line in quest_line_order:
    q = difflib.get_close_matches(que_line, os.listdir("html"))[0]
    a += li(button(q.split(".html")[0], id=q))
print(a)

<ul>
  <li>
    <button id="AndSoItBegins.html">AndSoItBegins</button>
  </li>
  <li>
    <button id="TipsandTricks.html">TipsandTricks</button>
  </li>
  <li>
    <button id="Tier0StoneAge.html">Tier0StoneAge</button>
  </li>
  <li>
    <button id="Tier05Steam.html">Tier05Steam</button>
  </li>
  <li>
    <button id="Tier1LV.html">Tier1LV</button>
  </li>
  <li>
    <button id="Tier2MV.html">Tier2MV</button>
  </li>
  <li>
    <button id="Tier3HV.html">Tier3HV</button>
  </li>
  <li>
    <button id="Tier4EV.html">Tier4EV</button>
  </li>
  <li>
    <button id="Tier5IV.html">Tier5IV</button>
  </li>
  <li>
    <button id="Tier6LuV.html">Tier6LuV</button>
  </li>
  <li>
    <button id="Tier7ZPM.html">Tier7ZPM</button>
  </li>
  <li>
    <button id="Tier8UV.html">Tier8UV</button>
  </li>
  <li>
    <button id="Tier9UHV.html">Tier9UHV</button>
  </li>
  <li>
    <button id="Tier10UEV.html">Tier10UEV</button>
  </li>
  <li>
    <button id="Tier11UIV.html">Tier11UIV</button>
  </li>
  <li>


In [4]:
str = """
        document.getElementById('%s').addEventListener('click', function () {
            document.getElementById('contentFrame').src = '%s';
        });
"""
for i in os.listdir("html"):
    if "html" in i:
        print(str % (i, "html/" + i))


        document.getElementById('AdeptThaumaturgy.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/AdeptThaumaturgy.html';
        });


        document.getElementById('AndSoItBegins.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/AndSoItBegins.html';
        });


        document.getElementById('AppliedEnergisti.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/AppliedEnergisti.html';
        });


        document.getElementById('BasicAutomation.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/BasicAutomation.html';
        });


        document.getElementById('BeeBreeding.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/BeeBreeding.html';
        });


        document.getEl